# AIVLE스쿨 2차 미니프로젝트: 신규 아파트 주차 수요 예측

<img src = "https://github.com/Jangrae/img/blob/master/parking.png?raw=true" width=800, align="left"/>

# 단계 3: 모델링

## [미션]

- 모델링을 수행합니다.
    - 전처리를 추가로 진행합니다.
    - 4개 이상의 알고리즘을 사용해 모델링을 수행합니다.
    - 각 모델에 대해 성능 튜닝을 수행합니다.
    - 성능을 비교해 최선의 모델을 선정합니다.
- 데이터 파이프라인 함수를 만듭니다.
- 새로운 데이터를 읽어와 예측을 수행합니다.

## 1. 환경설정

### (1) 로컬 수행(Anaconda)

- project 폴더에 필요한 파일들을 넣고, 본 파일을 열었다면, 별도 경로 지정이 필요하지 않습니다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 기본 경로
path = '/content/drive/MyDrive/2024.10.07_미니프로젝트 2차_실습자료/'

### (2) 구글 콜랩 수행

- 구글 콜랩을 사용중이면 구글 드라이브를 연결합니다.

In [3]:
# # 구글 드라이브 연결, 패스 지정
# import sys
# if 'google.colab' in sys.modules:
#     from google.colab import drive
#     drive.mount('/content/drive')
#     path = '/content/drive/MyDrive/project/'

### (3) 한글 폰트 표시용 라이브러리 설치

In [4]:
# 한글 표시를 위한 라이브러리 설치
!pip install koreanize_matplotlib -q

### (4) 라이브러리 불러오기

In [5]:
# 기본 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import koreanize_matplotlib
import seaborn as sns

# 모델링용 라이브러리 불러오기
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# 기타 라이브러리 불러기기
import joblib
import warnings

warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format='retina'

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


### (5) 데이터 불러오기

- joblib.dump() 함수를 사용해 base_data2.pkl 파일을 읽어옵니다.
- 읽어온 데이터를 data 데이터프레임으로 선언하고 기본 정보를 확인합니다.

In [6]:
# 파일 읽어오기
data = joblib.load(path+'base_data2.pkl')

# 확인
data.head()

,총세대수,준공연도,건물형태,난방방식,승강기설치여부,실차량수,총면적,면적10_30,면적30_40,면적40_50,면적50_60,면적60_70,면적70_80,면적80_200,임대보증금,임대료
0,78,2013,계단식,개별,1,109,6023.7683,0,0,0,78,0,0,0,5.696200e+07,642930.000000
1,35,2013,복도식,개별,1,35,1569.1668,35,0,0,0,0,0,0,6.306200e+07,470100.000000
2,88,2013,계단식,개별,1,88,7180.1396,0,0,0,88,0,0,0,7.219000e+07,586540.000000
3,477,2014,복도식,지역,1,943,47058.9273,0,0,0,150,0,216,111,1.015167e+08,950305.000000
4,15,2013,복도식,개별,1,21,543.0268,15,0,0,0,0,0,0,5.522750e+07,340148.333333


In [7]:
data.info()


<class 'pandas.core.frame.DataFrame'>
Index: 345 entries, 0 to 344
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   총세대수      345 non-null    int64  
 1   준공연도      345 non-null    object 
 2   건물형태      345 non-null    object 
 3   난방방식      345 non-null    object 
 4   승강기설치여부   345 non-null    object 
 5   실차량수      345 non-null    int64  
 6   총면적       345 non-null    float64
 7   면적10_30   345 non-null    int64  
 8   면적30_40   345 non-null    int64  
 9   면적40_50   345 non-null    int64  
 10  면적50_60   345 non-null    int64  
 11  면적60_70   345 non-null    int64  
 12  면적70_80   345 non-null    int64  
 13  면적80_200  345 non-null    int64  
 14  임대보증금     345 non-null    float64
 15  임대료       345 non-null    float64
dtypes: float64(3), int64(9), object(4)
memory usage: 53.9+ KB


In [8]:
data.describe()


,총세대수,실차량수,총면적,면적10_30,면적30_40,면적40_50,면적50_60,면적60_70,면적70_80,면적80_200,임대보증금,임대료
count,345.000000,345.000000,345.000000,345.000000,345.000000,345.000000,345.000000,345.000000,345.000000,345.000000,3.450000e+02,345.000000
mean,546.759420,555.431884,37282.661962,45.837681,169.347826,147.620290,132.889855,1.852174,16.895652,34.515942,2.382737e+07,197729.050210
std,432.359538,374.640474,29588.197927,161.133746,223.039796,216.584543,252.947082,32.771294,76.441995,116.975949,2.247481e+07,158760.777669
min,1.000000,21.000000,68.930000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
25%,178.000000,258.000000,11293.832100,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.272491e+07,112905.000000
50%,491.000000,498.000000,32190.360000,0.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.930850e+07,172000.000000
75%,853.000000,822.000000,56189.979600,0.000000,298.000000,264.000000,150.000000,0.000000,0.000000,0.000000,2.837700e+07,227564.000000
max,2289.000000,1657.000000,150126.863200,1181.000000,1429.000000,1258.000000,1444.000000,608.000000,756.000000,850.000000,1.898219e+08,950305.000000


### (6) 함수 생성

- 실젯값과 모델이 예측한 값을 시각화해 비교할 함수를 만듭니다.

In [9]:
# 실젯값, 예측값 비교 함수 만들기
# def model_plot(y_test, y_pred):
#     plt.figure(figsize=(10, 3))
#     plt.plot(y_test.values, label='Actual', linewidth=0.7, marker='o', markersize=2)
#     plt.plot(y_pred, label='Predicted', linewidth=0.7, marker='o', markersize=2)
#     plt.legend()
#     plt.show()

## 2. 모델링

- 모델링을 위한 결측치 처리, 데이터 분할, 스케일링, 가변수화 등을 수행합니다.
- 4개 이상의 알고리즘을 사용하여 모델을 만듭니다.
- 모델 이름은 서로 다르게 합니다. (예: model1, model2...)
- Linear Regression 이외의 알고리즘을 사용한 모델은 GridSearchCV() 함수로 성능 최적화를 수행합니다.
- 적절한 평가지표로 모델의 성능을 평가합니다.
- 실젯값과 예측값을 시각화해 비교합니다.
- 성능 비교를 통해 최선의 모델을 선정합니다.

### (1) 데이터 전처리

- 필요한 전처리를 수행합니다.

#### 1) 가변수화

- '건물형태', '난방방식' 변수에 대해 가변수화를 수행합니다.

In [11]:
dumn_cols = ['준공연도','건물형태', '난방방식', '승강기설치여부']
data = pd.get_dummies(data, columns=dumn_cols, drop_first = True, dtype=int)


#### 2) x, y 분리

- Target를 지정하고, x와 y로 분리합니다.

In [12]:
target = '실차량수'
x = data.drop(target, axis=1)
y = data[target]


#### 3) 학습용, 평가용 분리

- 학습용, 평가용 데이터를 적절한 비율로 분리합니다.

In [13]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


#### 4) 스케일링

- 필요한 경우 스케일링을 진행합니다.
- 예를 들어 KNN 알고리즘을 사용할 경우입니다.

In [14]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(x_train)
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)


### (2) 모델 1: KNN 알고리즘

In [31]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

# KNeighborsRegressor 하이퍼파라미터 그리드 설정
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],     # 이웃 수
    'weights': ['uniform', 'distance'],  # 가중치 방식
    'p': [1, 2]                          # 거리 계산 방식 (1: 맨해튼 거리, 2: 유클리드 거리)
}

# 모델 및 GridSearchCV 설정
model = KNeighborsRegressor()
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)

# GridSearchCV 모델 학습
grid_search.fit(x_train, y_train)

# 최적의 하이퍼파라미터 조합과 해당 성능 점수 확인
print("Best Parameters:", grid_search.best_params_)
print("Best CV Score (MSE):", -grid_search.best_score_)  # 음수로 반환되므로 양수로 변환

# 최적의 모델로 예측
best_model = grid_search.best_estimator_
y_pred = best_model.predict(x_test)

# 최적 모델 성능 평가
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('Optimized MSE:', mse)
print('Optimized R2:', r2)

result = []
result.append(['KNN', mse, r2])

Best Parameters: {'n_neighbors': 11, 'p': 1, 'weights': 'uniform'}
Best CV Score (MSE): 60373.19887088119
Optimized MSE: 82037.28518385433
Optimized R2: 0.45283073148685826


### (3) 모델 2: DecisionTree알고리즘


In [32]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

# 하이퍼파라미터 그리드 설정
param_grid = {
    'max_depth': [None, 5, 10, 15, 20],           # 트리의 최대 깊이
    'min_samples_split': [2, 5, 10, 20],          # 분할을 위한 최소 샘플 수
    'min_samples_leaf': [1, 2, 4, 8]              # 리프 노드의 최소 샘플 수
}

# 모델 및 GridSearchCV 설정
model2 = DecisionTreeRegressor(random_state=42)
grid_search = GridSearchCV(estimator=model2, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)

# GridSearchCV 모델 학습
grid_search.fit(x_train, y_train)

# 최적의 하이퍼파라미터 조합과 해당 성능 점수 확인
print("Best Parameters:", grid_search.best_params_)
print("Best CV Score (MSE):", -grid_search.best_score_)  # 음수로 반환되므로 양수로 변환

# 최적의 모델로 예측
best_model = grid_search.best_estimator_
y_pred = best_model.predict(x_test)

# 최적 모델 성능 평가
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('Optimized MSE:', mse)
print('Optimized R2:', r2)

result.append(['DecisionTree', mse, r2])

Best Parameters: {'max_depth': None, 'min_samples_leaf': 8, 'min_samples_split': 20}
Best CV Score (MSE): 37158.03705222707
Optimized MSE: 58911.77632572606
Optimized R2: 0.6070724002297747


### (4) 모델 3: LogisticRegression알고리즘

In [33]:
model3 = LogisticRegression(random_state=42)

model3.fit(x_train, y_train)
y_pred = model3.predict(x_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)


print('MSE:', mse)
print('R2:', r2)

result.append(['LogisticRegression', mse, r2])

MSE: 177180.73913043478
R2: -0.18175357957479044


### (5) 모델 4: RandomForest알고리즘

In [34]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

# 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [50, 100, 200],           # 트리의 개수
    'max_depth': [None, 10, 20, 30],          # 트리의 최대 깊이
    'min_samples_split': [2, 5, 10],          # 분할을 위한 최소 샘플 수
    'min_samples_leaf': [1, 2, 4]             # 리프 노드의 최소 샘플 수
}

# 모델 및 GridSearchCV 설정
model4 = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(estimator=model4, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)

# GridSearchCV 모델 학습
grid_search.fit(x_train, y_train)

# 최적의 하이퍼파라미터 조합과 해당 성능 점수 확인
print("Best Parameters:", grid_search.best_params_)
print("Best CV Score (MSE):", -grid_search.best_score_)  # 음수로 반환되므로 양수로 변환

# 최적의 모델로 예측
best_model = grid_search.best_estimator_
y_pred = best_model.predict(x_test)

# 최적 모델 성능 평가
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('Optimized MSE:', mse)
print('Optimized R2:', r2)

result.append(['RandomForest', mse, r2])

Best Parameters: {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200}
Best CV Score (MSE): 31125.01977066968
Optimized MSE: 47847.21805656166
Optimized R2: 0.6808703841707552


### (6) 모델 5: XGBOOST알고리즘

In [35]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

# 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [100, 200, 300],         # 트리의 개수
    'max_depth': [3, 5, 7],                  # 트리의 최대 깊이
    'learning_rate': [0.01, 0.1, 0.2],       # 학습률
    'subsample': [0.8, 1.0],                 # 각 트리에 사용할 샘플 비율
    'colsample_bytree': [0.8, 1.0]           # 각 트리에 사용할 피처의 비율
}

# 모델 및 GridSearchCV 설정
model5 = XGBRegressor(random_state=42)
grid_search = GridSearchCV(estimator=model5, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)

# GridSearchCV 모델 학습
grid_search.fit(x_train, y_train)

# 최적의 하이퍼파라미터 조합과 해당 성능 점수 확인
print("Best Parameters:", grid_search.best_params_)
print("Best CV Score (MSE):", -grid_search.best_score_)  # 음수로 반환되므로 양수로 변환

# 최적의 모델로 예측
best_model = grid_search.best_estimator_
y_pred = best_model.predict(x_test)

# 최적 모델 성능 평가
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('Optimized MSE:', mse)
print('Optimized R2:', r2)

result.append(['XGBoost', mse, r2])

Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 300, 'subsample': 0.8}
Best CV Score (MSE): 30734.33067923048
Optimized MSE: 51167.06237859615
Optimized R2: 0.6587278246879578


### (6) 모델 6: LGBM알고리즘

In [36]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

# 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [100, 200, 300],         # 트리의 개수
    'max_depth': [3, 5, 7, -1],              # 트리의 최대 깊이 (-1은 제한 없음)
    'learning_rate': [0.01, 0.1, 0.2],       # 학습률
    'num_leaves': [31, 50, 70],              # 하나의 트리에서 사용할 수 있는 리프의 수
    'subsample': [0.8, 1.0],                 # 각 트리에 사용할 샘플 비율
    'colsample_bytree': [0.8, 1.0]           # 각 트리에 사용할 피처의 비율
}

# 모델 및 GridSearchCV 설정
model = LGBMRegressor(random_state=42)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)

# GridSearchCV 모델 학습
grid_search.fit(x_train, y_train)

# 최적의 하이퍼파라미터 조합과 해당 성능 점수 확인
print("Best Parameters:", grid_search.best_params_)
print("Best CV Score (MSE):", -grid_search.best_score_)  # 음수로 반환되므로 양수로 변환

# 최적의 모델로 예측
best_model = grid_search.best_estimator_
y_pred = best_model.predict(x_test)

# 최적 모델 성능 평가
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('Optimized MSE:', mse)
print('Optimized R2:', r2)

result.append(['LGBM', mse, r2])

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000073 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 534
[LightGBM] [Info] Number of data points in the train set: 276, number of used features: 15
[LightGBM] [Info] Start training from score 560.601449
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

### (7) 성능 비교

- 각 모델의 성능을 비교합니다.

In [37]:
print(result)


[['KNN', 82037.28518385433, 0.45283073148685826], ['DecisionTree', 58911.77632572606, 0.6070724002297747], ['LogisticRegression', 177180.73913043478, -0.18175357957479044], ['RandomForest', 47847.21805656166, 0.6808703841707552], ['XGBoost', 51167.06237859615, 0.6587278246879578], ['LGBM', 50914.23212398011, 0.660414126506598]]


## 3. 파이프라인 구축

- 새로운 데이터를 불러오고, 이 데이터를 처리할 파이프라인 함수를 만듭니다.

### (1) New Data 불러오기

- test.xlsx 파일을 읽어와 new_data 데이터프레임으로 선언합니다.
- 해당 데이터는 '실차량수' 변수가 없는 것 외에는, 최초 데이터와 동일한 구조입니다.
- 이 데이터를 대상으로 전처리와 예측을 수행합니다.

In [38]:
# 파일 읽어오기
new_data = pd.read_excel(path+'test.xlsx')

# 확인
new_data.head()

,단지코드,단지명,총세대수,전용면적별세대수,지역,준공일자,건물형태,난방방식,승강기설치여부,단지내주차면수,전용면적,공용면적,임대보증금,임대료
0,C0005,서울석촌 도시형주택(공임10년),20,6,서울,20121115,복도식,개별가스난방,전체동 설치,9,17.53,11.7251,50449000,263710
1,C0005,서울석촌 도시형주택(공임10년),20,10,서울,20121115,복도식,개별가스난방,전체동 설치,9,24.71,16.5275,52743000,321040
2,C0005,서울석촌 도시형주택(공임10년),20,4,서울,20121115,복도식,개별가스난방,전체동 설치,9,26.72,17.8720,53890000,332510
3,C0017,대구혁신센텀힐즈,822,228,대구경북,20180221,계단식,지역난방,NaN,824,51.87,20.9266,29298000,411200
4,C0017,대구혁신센텀힐즈,822,56,대구경북,20180221,계단식,지역난방,NaN,824,59.85,24.1461,38550000,462600


### (2) 데이터 파이프라인 구축

- 데이터 파이프라인 함수를 만듭니다.
- 학습 데이터에 대해 진행했던 모든 전처리 과정을 평가 데이터에도 일괄 진행해야 합니다.
    - 입력: new_data
    - 출력: 전처리가 완료된 예측 직전 데이터프레임
- 새로운 데이터에는 '실차량수' 변수가 없음을 유의합니다.
- 참고: 다음 내용들이 처리되어야 합니다.
    - 결측치 처리
    - 변수 추가
    - 불필요한 변수 제거
    - 단지 데이터, 상세 데이터 분리
    - 단지코드별 총면적 합 집계
    - 전용면적 구간별 집계 (피벗 형태)
    - 임대보증금, 임대료 평균 집계
    - 집계 결과 병합
    - 난방방식: 개별, 지역, 중앙 세 가지로 묶기
    - 승강기설치여부: 0, 1 값으로 변경
    - 단지모드, 지역 변수 제거
    - 가변수화

In [41]:
# 파이프라인 함수 정의
def data_pipeline(data):
    # 데이터 복사
    apt01 = data.copy()

    # 결측치 처리: '건물형태', '난방방식', '승강기설치여부'의 결측치는 최빈값으로 채우기
    nan_cols = ['건물형태', '난방방식', '승강기설치여부']
    for col in nan_cols:
        apt01[col] = apt01[col].fillna(apt01[col].mode()[0])

    # 변수 추가: '총면적' 계산
    apt01['총면적'] = (apt01['전용면적'] + apt01['공용면적']) * apt01['전용면적별세대수']

    # 불필요한 변수 제거
    apt01 = apt01.drop(['단지내주차면수', '단지명', '준공일자'], axis=1)

    # 단지코드별 '총면적' 합 집계
    df_area = apt01.groupby('단지코드')['총면적'].sum().reset_index()

    # 전용면적 구간별 집계 (피벗 형태)
    apt01['전용면적구간'] = pd.cut(apt01['전용면적'], bins=[10, 30, 40, 50, 60, 70, 80, 200],
                                   labels=['10-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-200'])
    df_pivot = apt01.pivot_table(index='단지코드', columns='전용면적구간', values='전용면적별세대수', aggfunc='sum').reset_index()

    # 임대보증금, 임대료 평균 집계
    df_rent = apt01.groupby('단지코드')[['임대보증금', '임대료']].mean().reset_index()

    # 집계 결과 병합
    base_data = pd.merge(apt01, df_area, on='단지코드', how='left')
    base_data = pd.merge(base_data, df_pivot, on='단지코드', how='left')
    base_data = pd.merge(base_data, df_rent, on='단지코드', how='left')

    # 난방방식: 개별, 지역, 중앙으로 묶기
    base_data['난방방식'] = base_data['난방방식'].replace({
        '개별가스난방': '개별', '개별유류난방': '개별',
        '지역난방': '지역', '지역가스난방': '지역', '지역유류난방': '지역',
        '중앙가스난방': '중앙', '중앙난방': '중앙', '중앙유류난방': '중앙'
    })

    # 승강기설치여부: 0과 1로 변경
    base_data['승강기설치여부'] = base_data['승강기설치여부'].apply(lambda x: 1 if x == '설치' else 0)

    # 범주형 변수 가변수화
    base_data = pd.get_dummies(base_data, columns=['난방방식', '건물형태'])

    # 결과 반환 (전처리가 완료된 예측 직전 데이터프레임)
    return base_data

# 함수 사용 예시
# processed_data = data_pipeline(new_data)


### (3) 예측하기

- new_data를 파이프라인을 사용해 전처리한 후 가장 성능이 좋았던 모델로 예측한 결과를 확인합니다.

In [42]:
# 데이터 전처리
data = data_pipeline(new_data)

# 확인
data.head()

,단지코드,총세대수,전용면적별세대수,지역,승강기설치여부,전용면적,공용면적,임대보증금_x,임대료_x,총면적_x,...,70-80,80-200,임대보증금_y,임대료_y,난방방식_개별,난방방식_중앙,난방방식_지역,건물형태_계단식,건물형태_복도식,건물형태_혼합식
0,C0005,20,6,서울,0,17.53,11.7251,50449000,263710,175.5306,...,0,0,5.236067e+07,305753.333333,True,False,False,False,True,False
1,C0005,20,10,서울,0,24.71,16.5275,52743000,321040,412.3750,...,0,0,5.236067e+07,305753.333333,True,False,False,False,True,False
2,C0005,20,4,서울,0,26.72,17.8720,53890000,332510,178.3680,...,0,0,5.236067e+07,305753.333333,True,False,False,False,True,False
3,C0017,822,228,대구경북,0,51.87,20.9266,29298000,411200,16597.6248,...,0,0,3.546600e+07,445466.666667,False,False,True,True,False,False
4,C0017,822,56,대구경북,0,59.85,24.1461,38550000,462600,4703.7816,...,0,0,3.546600e+07,445466.666667,False,False,True,True,False,False


In [49]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# model4 다시 정의
model7 = RandomForestRegressor(random_state=42)

# 모델 학습
model7.fit(x_train, y_train)

# 예측하기
predicted = model7.predict(data)

# 예측 결과 확인
print(predicted)


ValueError: Cannot cast object dtype to float32

- 아파트 기본 정보에 예측한 차량수를 붙여 마무리합니다.

In [ ]:
# 데이터 셋 두개로 나누기
vars = ['단지코드', '단지명', '총세대수', '지역', ]
result = new_data[vars].copy()
result = result.drop_duplicates()
result.reset_index(drop=True, inplace=True)

# 예측 결과 추가
result['예상차량수'] = predicted.round(1).astype(int)

# 확인
result